In [1]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [17]:
import addict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
from tqdm.notebook import tqdm

import celeri
celeri = reload(celeri)

# Plotting the global model is much much faster with tex fonts turned off
plt.rcParams['text.usetex'] = False

In [15]:
command_file_name = "./data/western_north_america/basic_command.json"
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure, block = celeri.assign_block_labels(segment, station, block, mogi, sar)
# celeri.plot_block_labels(segment, block, station, closure)

In [4]:
assembly = addict.Dict()
operators = addict.Dict()
assembly = celeri.merge_geodetic_data(assembly, station, sar)
assembly, operators.block_motion_constraints = celeri.block_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.slip_rate_constraints(assembly, segment, block, command)
operators.okada_segment_station = celeri.get_segment_station_operator_okada(segment, station, command)
operators.slip_rate_segment_block = celeri.get_fault_slip_rate_partials(segment, block)
assembly.index.sz_elastic = operators.okada_segment_station.shape # Not sure this is correct
assembly.index.sz_slip = operators.slip_rate_segment_block.shape # Not sure this is correct
operators.block_rotation = celeri.get_block_rotation_operator(station, block)
assembly.index.sz_rotation = operators.block_rotation.shape # Not sure this is correct
assembly = celeri.station_row_keep(assembly) # Not sure this is correct


Calculating Okada partials for segments:   0%|          | 0/837 [00:00<?, ?it/s]

In [5]:
# celeri.plot_segment_displacements(segment, station, command, segment_idx=0, strike_slip=1, dip_slip=0, tensile_slip=0, lon_min=235, lon_max=255, lat_min=30, lat_max=50, quiver_scale=1e-1)

In [25]:
model = addict.Dict()

# function [strain, Index, Model] def get_strain_operator(block, station, segment, command, assembly)
"""
Wrapper function for calculating strain partial derivatives.
"""

strain = np.empty(len(station))
assembly.index.strain_block = np.empty(0)
model.lon_strain = np.zeros(len(block))
model.lat_strain = np.zeros(len(block))

if any(block.strain_rate_flag): # If any blocks have strain turned on
    pass
    if command.strain_method == 1:
        pass
#          [strain, Index.strainBlock,...
#           Model.lonStrain, Model.latStrain]   = GetStrainCentroidPartials(Block, Station, Segment);
#    if command.strain_method == 2: # Solve for the reference coordinates
#          [strain, Index.strainBlock]          = GetStrain56Partials(Block, Station, Segment);
#    if command.strain_method == 3: # Solve for the reference latitude only
#          [strain, Index.strainBlock]          = GetStrain34Partials(Block, Station, Segment);
#    if command.strain_method == 4: # Use the simulated annealing approach to solve for reference coordinates inside the block boundaries
#          [strain, Index.strainBlock]          = GetStrainSearchPartials(Block, Station, Segment);
#    end      

def get_strain_rate_centroid_operator(block, station, segment):
    """
    Calculate strain partial derivatives assuming a strain centroid at the center of each block
    """
    n_stations = len(station)
    n_blocks = len(block)
    G = np.zeros((3 * n_stations, 3 * n_blocks))  # TODO: Redo so that it's only for the blocks with internal strain
    earth_radius_mm = celeri.RADIUS_EARTH * celeri.M2MM # radius of Earth in mm

    # Convert station positions into radians and co-latitude
    station_lon = np.deg2rad(station.lon)
    station_lat = station.lat.to_numpy()
    station_lat[np.where(station_lat >= 0)[0]] = 90.0 - station_lat[np.where(station_lat >= 0)[0]]
    station_lat[np.where(station_lat < 0)[0]] = -90.0 - station_lat[np.where(station.lat < 0)[0]]
    station_lat = np.deg2rad(station_lat)

    # Create array containing column index limits of the design matrix
    first_column_idx = 3 * station.block_label
    last_column_idx = 3 * station.block_label + 3
    # block_centroid_lon = np.zeros(n_blocks)
    # block_centroid_lat = np.zeros(n_blocks)

    for i in range(len(station)):
        if block.strain_rate_flag[station.block_label[i]] == 1: # what does this line mean????
            # Partials with block centroid
            # The block "centroid" is only an approximation given by the mean of its coordinates
            # This should work reasonably well for the "chopped" case but is not exact or general
            idx = np.union(np.where(segment.east_label == station.block_label[i])[0], np.where(segment.west_label == station.block_label[i])[0])
            lon0  = np.mean(np.concatenate(segment.lon1[idx], segment.lon2[idx]))
            lat0  = np.mean(np.concatenate(segment.lat1[idx], segment.lat2[idx]))
            # block_centroid_lon[station.block_label[i]] = lon0
            # block_centroid_lat[station.block_label[i]] = lat0
            lon0 = np.deg2rad(lon0)
            if lat0 >= 0:
                lat0 = 90.0 - lat0
            elif lat0 < 0:
                lat0 = -90.0 - lat0
            lat0 = np.deg2rad(lat0)
            G[3 * i:3 * i + 3, first_column_idx[i]:last_column_idx[i]] = np.array([[earth_radius_mm * (station_lon[i] - lon0) * np.sin(lat0), earth_radius_mm * (station_lat[i] - lat0), 0], 
                       [0, earth_radius_mm * (station_lon[i] - lon0) * np.sin(lat0), earth_radius_mm * (station_lat[i] - lat0)],
                       [0, 0, 0]])

    # TODO: This should become command toggle or just throw a warning rather than doing it automatically
    # Write a test here that checks the number of stations on each block that includes
    # internal strain.  Send an error if any of those blocks have less the N_STATIONS_STRAIN_RATE_BLOCK stations on them
    # Get rid of blocks that don't have enough stations on them
    # for i = 1:size(G, 2)/3;
    #    j                                        = 3*i-1;
    #    numSta(i)                                = numel(find(G(:, j)));
    # end
    # toDelete                                    = [3*find(numSta<2*minSta)-2 3*find(numSta<2*minSta)-1 3*find(numSta<2*minSta)];
    # strainBlockIdx                              = setdiff(1:1:nBlocks*3, toDelete);
    # G(:,toDelete)                               = [];
    # return G, strainBlockIdx, block, toDelete
    strain_rate_block_idx = np.where(block.strain_rate_flag.to_numpy() > 0)[0]
    return G, strain_rate_block_idx

G, strain_rate_block_idx = get_strain_rate_centroid_operator(block, station, segment)
print(strain_rate_block_idx)

[]


In [23]:
G.shape
print(np.min(G), np.max(G))
plt.figure()
plt.imshow(G, aspect="auto")
plt.show()

0.0 0.0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …